In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = './전처리데이터_결합.csv'

In [ ]:
#Q-GIS 에서 작업한 전처리 데이터 읽어오기

pre_process_set = pd.read_csv('./전처리데이터_결합.csv', encoding = 'cp949' engine = 'python')
pre_process_set = pre_process_set.sort_values(by='gid')

pre_process_set.fillna(0, inplace = True)

In [ ]:
weight_column = ['count','건물1','건물2','건물3','val','sumPOP_10']

#가중치 부여
#count : 학교, 가중치 4 부여
#건물 1 : 가중치 1점 건물의 집합
#건물 2: 가중치 2점 건물의 집합
#건물 3: 가중치 3점 건물의 집합
#val : 청소년 생산인구, 가중치 1 부여

weight = [4,1,2,3,1,1]

In [ ]:

# Min-Max 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

norm = pre_process_set.copy()
norm.fillna(0,inplace = True)
norm.set_index('gid', inplace = True)

norm[weight_column] = scaler.fit_transform(norm.loc[:,점수칼럼])
norm.reset_index(inplace = True)

In [ ]:
# 정규화 결과를 활용한 총점 계산 및 격자별 순위 산정

df = norm.copy()

for i in range (0, len(weight_column)):
    name = weight_column[i]
    df[name] = df[name] * weight[i]

df.set_index(['gid','count','경도','위도'], inplace = True)

df['sum'] = df.sum(1)
df.reset_index(inplace = True)

df['rank'] = df['sum'].rank(method = 'dense', ascending = False).astype(int)

final = df.sort_values(by='rank')

In [ ]:
# folium 을 활용한 시각화

import folium

for i in range (5):
    show_map = folium.Map(location = [final.loc[i,'위도'], final.loc[i,'경도']], zoom_start = 15)
    folium.Marker([final.loc[i,'위도'], final.loc[i,'경도']], add_to(show_map))
    print(final.loc[i,'위도'], final.loc[i,'경도'])